In [1]:
import pandas as pd
import surprise
import numpy as np

위에 있는 패키지는 Similarity를 구하기 위해 필요한 패키지들입니다.

In [2]:
# 데이터 읽기
import os.path
import csv

list = []
movie_code = 0
search = ":"

with open('combined_data_1.txt','r') as textfile:
    for row in textfile:
        row_split = row.split()
        for element in row_split:
            element_split = element.split(',')
            check = element_split[0]
            result = check.find(search)
            check = check.replace(":","")
            if result == -1:
                element_split.append(movie_code)
                list.append(element_split)
            else:
                movie_code = check

In [3]:
list[:2]

[['1488844', '3', '2005-09-06', '1'], ['822109', '5', '2005-05-13', '1']]

In [4]:
len(list)

24053764

In [5]:
# list를 dataframe으로
data = pd.DataFrame(list)
data.shape

(24053764, 4)

In [6]:
# 필요없는 column 지우기(Rating한 날짜)
data.drop(2, axis = 1, inplace = True)

# 계산 편하게 하기 위해 int형으로 형 변환
data[3] = data[3].apply(lambda x: int(x))

In [7]:
# 데이터가 24053764개이기 때문에 영화는 300개 이내, 유저수는 1000으로 수정 
data = data[data[3] < 300]
data.shape

(1453857, 3)

In [8]:
data.head()

,0,1,3
0,1488844,3,1
1,822109,5,1
2,885013,4,1
3,30878,4,1
4,823519,3,1


- 위 데이터는 User-ID, Rating, Movie 순입니다.

In [9]:
# 유저수 1000으로 줄이기
x = data[0].unique()[:1000]

def find_ID(data):
    return True if data[0] in x else False

data = data[data.apply(find_ID, axis = 1)]
print(data.shape)
data.head()

(15124, 3)


,0,1,3
0,1488844,3,1
1,822109,5,1
2,885013,4,1
3,30878,4,1
4,823519,3,1


- 위의 데이터로 ID에 따른 영화별 Rating값을 데이터 프레임으로 만들기

In [10]:
# ID의 값을 편하게 하기 위해 int형으로 변환
df = pd.DataFrame(data[0].unique())
df[0] = df[0].apply(lambda x : int(x))

In [11]:
# 영화별 Rating값을 nan값으로 초기화
df = pd.concat([df,pd.DataFrame([[np.nan]*299]*1000, columns = [i+1 for i in range(299)])], axis = 1)

# column 이름 변경 & 정렬 & index로 교체
df.rename(columns = {0 : 'ID'}, inplace = True)
df = df.sort_values(by = 'ID').set_index('ID')
df.head(2)

,1,2,3,4,5,6,7,8,9,10,...,290,291,292,293,294,295,296,297,298,299
ID,,,,,,,,,,,,,,,,,,,,,
915,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
def Enter_value(data):
    df[data[3]][int(data[0])] = int(data[1])

x = data.apply(Enter_value, axis = 1)
df.head(2)

,1,2,3,4,5,6,7,8,9,10,...,290,291,292,293,294,295,296,297,298,299
ID,,,,,,,,,,,,,,,,,,,,,
915,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1333,NaN,NaN,4.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [13]:
def EUC(u, v):
    A =  np.sum(np.square(u - v))
    B = np.sqrt(A)
    return B
#두 리스트에 대한 유클리드 거리입니다.

def EUC_array(a):
    a = np.array(pd.DataFrame(a).fillna(False))
    number_of_user = np.size(a, axis = 0)
    N = np.full((number_of_user, number_of_user), 0.0)
    #배열 a의 크기에 맞게 빈 배열을 만듭니다.
    for u in range(0, number_of_user):
        for v in range(u, number_of_user):
            A =  np.sum(np.square(a[u,] - a[v,]))
            N[v, u] = np.sqrt(A)
            N[u, v] = N[v, u]
    return N
#하나의 배열에 대한 유클리드 거리입니다.

In [14]:
pd.DataFrame(EUC_array(df.values)).head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.000000,19.798990,8.366600,23.302360,12.569805,11.357817,13.490738,9.110434,6.164414,13.228757,...,26.645825,17.058722,13.379088,9.539392,12.922848,9.746794,13.000000,14.966630,15.362291,7.681146
1,19.798990,0.000000,19.183326,25.826343,19.949937,18.357560,20.928450,19.313208,18.920888,19.209373,...,26.343880,24.596748,21.000000,17.860571,20.074860,19.773720,18.411953,17.776389,20.591260,19.157244
2,8.366600,19.183326,0.000000,23.302360,12.649111,10.246951,14.142136,9.746794,10.000000,12.369317,...,26.795522,17.860571,13.453624,9.539392,12.688578,10.630146,13.674794,14.764823,15.427249,9.539392
3,23.302360,25.826343,23.302360,0.000000,24.351591,23.452079,24.556058,23.021729,23.937418,22.538855,...,27.331301,23.916521,24.000000,23.494680,24.819347,24.859606,24.939928,23.685439,24.474477,22.803509
4,12.569805,19.949937,12.649111,24.351591,0.000000,13.820275,15.748016,13.747727,13.266499,15.198684,...,26.076810,20.124612,15.968719,14.387495,14.525839,14.525839,14.035669,14.832397,15.231546,14.387495


In [15]:
def COS(u,v):
    return np.dot(u,v)/(np.linalg.norm(u) * np.linalg.norm(v))
#두 리스트에 대한 COS를 구하는 알고리즘입니다.

def COS_array(a):
    a = np.array(pd.DataFrame(a).fillna(False))
    number_of_user = np.size(a, axis = 0)
    N = np.full((number_of_user, number_of_user), 0.0)
    #배열 a의 크기에 맞게 빈 배열을 만듭니다.
    for u in range(0, number_of_user):
        for v in range(0, number_of_user):
            m = COS(a[u, ],a[v, ])
            N[u,v] = m
            N[v,u] = N[u, v]
    return N
#하나의 배열에 대한 COS를 구하는 알고리즘입니다.

In [17]:
pd.DataFrame(COS_array(df.values))

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,0.065218,0.304981,0.203859,0.249280,0.000000,0.000000,0.276079,0.630295,0.236790,...,0.145607,0.381137,0.234114,0.000000,0.242429,0.000000,0.000000,0.000000,0.203753,0.353381
1,0.065218,1.000000,0.197078,0.290136,0.262457,0.328305,0.153860,0.203009,0.231057,0.340323,...,0.381556,0.144377,0.203451,0.361725,0.263346,0.106552,0.359879,0.451468,0.303056,0.184553
2,0.304981,0.197078,1.000000,0.219278,0.290926,0.291442,0.000000,0.276172,0.152542,0.394784,...,0.145657,0.282419,0.273226,0.165703,0.323348,0.000000,0.000000,0.115486,0.237792,0.165703
3,0.203859,0.290136,0.219278,1.000000,0.215075,0.228923,0.180851,0.263009,0.137049,0.367099,...,0.420853,0.379078,0.261956,0.184602,0.186286,0.000000,0.130264,0.289480,0.274680,0.279881
4,0.249280,0.262457,0.290926,0.215075,1.000000,0.229706,0.168028,0.188111,0.207804,0.302514,...,0.291683,0.199099,0.239277,0.000000,0.346884,0.000000,0.310383,0.353978,0.416492,0.000000
5,0.000000,0.328305,0.291442,0.228923,0.229706,1.000000,0.314208,0.464830,0.083269,0.210115,...,0.246482,0.121406,0.095880,0.316587,0.124107,0.000000,0.226134,0.260044,0.166892,0.135680
6,0.000000,0.153860,0.000000,0.180851,0.168028,0.314208,1.000000,0.198497,0.000000,0.255373,...,0.235552,0.091344,0.284049,0.238197,0.163409,0.573012,0.223309,0.217888,0.219744,0.178647
7,0.276079,0.203009,0.276172,0.263009,0.188111,0.464830,0.198497,1.000000,0.138086,0.214423,...,0.307657,0.153393,0.106000,0.150000,0.109764,0.000000,0.135417,0.104542,0.092253,0.150000
8,0.630295,0.231057,0.152542,0.137049,0.207804,0.083269,0.000000,0.138086,1.000000,0.266479,...,0.092249,0.388327,0.273226,0.000000,0.121255,0.000000,0.000000,0.000000,0.169852,0.294584
9,0.236790,0.340323,0.394784,0.367099,0.302514,0.210115,0.255373,0.214423,0.266479,1.000000,...,0.438220,0.320686,0.704591,0.428845,0.417763,0.206328,0.134014,0.319429,0.425295,0.128654


In [18]:
def PCC(u, v):
    U_mean = np.nanmean(np.where(u!=0,np.nan))
    V_mean = np.nanmean(np.where(v!=0,np.nan))
    
    zero_u = np.where(u == 0)
    zero_v = np.where(v == 0)
    zero = np.concatenate((zero_u, zero_v), axis =None)
    
    U = np.delete(u, zero_u)
    V = np.delete(v, zero_v)
    U = U - U_mean
    V = V - V_mean
    
    return np.dot(U, V)/(np.linalg.norm(U) * np.linalg.norm(V))
#두 리스트(NULL 부분 제외)에 대한 PCC를 구하는 알고리즘입니다.

def PCC_array(a):
    number_of_user = np.size(a, axis = 0)
    N = np.full((number_of_user, number_of_user), 0.0)
    #배열 a의 크기에 맞게 빈 배열을 만듭니다.
    mean = np.nanmean(np.where(a!=0, a, np.nan), axis=1)
    
    for u in range(0, number_of_user):
        for v in range(u, number_of_user):
            zero_u = np.where(a[u, ] == 0)
            zero_v = np.where(a[v, ] == 0)
            zero = np.concatenate((zero_u, zero_v), axis = None)
      
            U = np.delete(a[u, ], zero)
            V = np.delete(a[v, ], zero)
            U = U - mean[u]
            V = V - mean[v]
            
            U[np.isnan(U)] = 0
            V[np.isnan(V)] = 0
            
            Inner = np.dot(U,V)
            NormU = np.linalg.norm(U)
            NormV = np.linalg.norm(V)
            N[u, v] = Inner / (NormU * NormV)
            N[v, u] = N[u, v]
            
    N[np.isnan(N)] = -1
    return N
#하나의 배열(NULL 부분 제외)에 대한 PCC를 구하는 알고리즘입니다.

In [19]:
pd.DataFrame(PCC_array(df.values))

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in double_scalars


,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,0.088411,-0.258199,-7.267524e-02,0.031944,0.000000,0.000000,-0.059761,-0.645497,0.131590,...,0.088302,0.000000,-1.118034e-01,0.000000,-0.654654,0.000000,0.000000,0.000000e+00,0.072524,-0.353553
1,0.088411,1.000000,-0.061566,5.516675e-02,0.125634,-0.058732,-0.038738,-0.265780,-0.164636,-0.040895,...,0.082800,-0.056335,2.156655e-02,0.000000,-0.021047,0.015468,-0.095500,1.625094e-02,0.050074,-0.143977
2,-0.258199,-0.061566,1.000000,-2.501955e-02,0.008248,-0.024398,0.000000,0.061721,0.133333,-0.033976,...,-0.091198,0.009281,-2.886751e-02,0.182574,0.422577,0.000000,0.000000,-5.729899e-02,0.018726,0.182574
3,-0.072675,0.055167,-0.025020,1.000000e+00,0.024763,-0.022891,-0.093823,-0.020268,0.062549,0.062162,...,0.054548,-0.036572,-3.791825e-02,-0.051389,0.026432,0.000000,0.056317,2.687991e-03,-0.008534,0.017130
4,0.031944,0.125634,0.008248,2.476294e-02,1.000000,-0.072442,0.090726,-0.034362,-0.486624,0.130308,...,0.238350,-0.146969,1.214286e-01,0.000000,-0.027883,0.000000,0.196615,1.748598e-01,0.386558,0.000000
5,0.000000,-0.058732,-0.024398,-2.289055e-02,-0.072442,1.000000,0.048795,0.180702,0.243975,0.478714,...,-0.008344,0.044153,2.112886e-02,-0.133631,-0.020620,0.000000,-0.006276,-7.339249e-02,-0.027412,0.066815
6,0.000000,-0.038738,0.000000,-9.382333e-02,0.090726,0.048795,1.000000,0.092582,0.000000,0.067953,...,0.068399,-0.162413,8.660254e-02,0.091287,0.028172,0.149071,0.040016,4.297424e-02,0.037451,-0.091287
7,-0.059761,-0.265780,0.061721,-2.026816e-02,-0.034362,0.180702,0.092582,1.000000,0.030861,-0.031456,...,0.047494,0.008592,1.336306e-02,0.042258,0.078246,0.000000,0.096589,-2.873463e-02,-0.008668,0.042258
8,-0.645497,-0.164636,0.133333,6.254889e-02,-0.486624,0.243975,0.000000,0.030861,1.000000,0.144400,...,-0.182396,0.129931,-2.886751e-02,0.000000,0.338062,0.000000,0.000000,0.000000e+00,-0.254134,0.273861
9,0.131590,-0.040895,-0.033976,6.216183e-02,0.130308,0.478714,0.067953,-0.031456,0.144400,1.000000,...,0.079886,-0.078044,2.133276e-01,-0.069786,-0.136398,0.075974,-0.081577,-9.125708e-02,-0.002727,-0.093048
